In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Modification

In [1]:
import os
import shutil
import random

# Set the seed for reproducibility
random.seed(42)

# Specify the directory containing the initial dataset
source_directory = '/kaggle/input/vision/256_ObjectCategories'

# Specify the base directory for the train/test split
base_directory = '/kaggle/working/vision/256_ObjectCategories'
train_dir = os.path.join(base_directory, 'train')
test_dir = os.path.join(base_directory, 'test')

# Ratio of data to be used for training (the rest will be for testing)
train_ratio = 0.8

# Create train and test directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Process each subdirectory in the source directory
for class_name in os.listdir(source_directory):
    class_dir = os.path.join(source_directory, class_name)
    
    # Only process directories
    if os.path.isdir(class_dir):
        # Create corresponding directories in train and test folders
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)
        
        # Get a list of all files in the class directory
        files = [file for file in os.listdir(class_dir) if file.endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(files)  # Shuffle to ensure random distribution
        
        # Calculate split point
        split_point = int(len(files) * train_ratio)
        
        # Distribute files into the train and test directories
        for i, file in enumerate(files):
            src = os.path.join(class_dir, file)
            if i < split_point:
                dest = os.path.join(train_dir, class_name, file)
            else:
                dest = os.path.join(test_dir, class_name, file)
            shutil.copy(src, dest)  # Use copy instead of move

print("Data successfully split into train and test directories.")


Data successfully split into train and test directories.


# Model

In [2]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder

def load_dataset(data_dir, transform, exclude='257.clutter'):
    # Initialize the dataset using ImageFolder
    dataset = ImageFolder(data_dir, transform=transform)
    # Filter out the unwanted folder
    filtered_samples = [s for s in dataset.samples if exclude not in s[0]]
    dataset.samples = filtered_samples
    dataset.imgs = filtered_samples  # torchvision 0.2.1 compatibility
    dataset.targets = [s[1] for s in filtered_samples]  # update the targets accordingly
    return dataset

# Define transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Path to your dataset
data_dir = '/kaggle/working/vision/256_ObjectCategories'

# Load datasets excluding the '257.clutter' folder
train_dataset = load_dataset(os.path.join(data_dir, 'train'), transform)
test_dataset = load_dataset(os.path.join(data_dir, 'test'), transform)

# Data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# Load a pretrained ResNet and modify it
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # Freeze all parameters initially

# Unfreeze some layers
for param in model.layer4.parameters():
    param.requires_grad = True  # Unfreeze the last layer block

# Adjust the final fully connected layer for 256 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 256)  # Adjust output to 256 classes, excluding the clutter

# Move model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)

In [6]:
import time
def train_model(model, criterion, optimizer, num_epochs=25):
    model.train()
    for epoch in range(num_epochs):
        start_time = time.time()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        end_time = time.time()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Run Time: {end_time-start_time:.4f}')
start = time.time()
train_model(model, criterion, optimizer, num_epochs=10)
end = time.time()
print("Elapsed time: ", end-start)

Epoch 1/10, Loss: 2.8639, Run Time: 193.4562
Epoch 2/10, Loss: 1.1082, Run Time: 192.8813
Epoch 3/10, Loss: 0.7097, Run Time: 193.1946
Epoch 4/10, Loss: 0.4901, Run Time: 193.1559
Epoch 5/10, Loss: 0.3441, Run Time: 193.3544
Epoch 6/10, Loss: 0.2447, Run Time: 194.4084
Epoch 7/10, Loss: 0.1844, Run Time: 193.9912
Epoch 8/10, Loss: 0.1510, Run Time: 193.9216
Epoch 9/10, Loss: 0.1268, Run Time: 193.6822
Epoch 10/10, Loss: 0.1035, Run Time: 188.2180
Elapsed time:  1930.266622543335


In [7]:
def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Test Accuracy: {100 * correct / total:.2f}%')

evaluate_model(model)

Test Accuracy: 78.60%
